In [1]:
def read_bio_file(file_path):
    sentences = []
    sentence = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:  # new sentence
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                token, label = line.split('\t')
                sentence.append((token, label))
    if sentence:
        sentences.append(sentence)
    return sentences


def convert_to_spacy_format(sentences):
    spacy_data = []
    for sentence in sentences:
        text = ' '.join([token for token, label in sentence])
        entities = []
        idx = 0
        for token, label in sentence:
            start_idx = idx
            end_idx = idx + len(token)
            if label.startswith('B-'):
                entities.append([start_idx, end_idx, label.split('B-')[1]])
            elif label.startswith('I-'):
                entities[-1][1] = end_idx
            idx += len(token) + 1  # +1 for space
        spacy_data.append((text, {"entities": entities}))
    return spacy_data


file_path = 'restaurant_restauranttrain.bio'
sentences = read_bio_file(file_path)
TRAIN_DATA = convert_to_spacy_format(sentences)

In [2]:
import spacy
from spacy.training.example import Example
from tqdm import tqdm
import random

nlp = spacy.blank('en')
ner = nlp.add_pipe('ner', last=True)

# Add labels to the NER pipe
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Train the model
optimizer = nlp.begin_training()
for itn in range(2):  # Assume 2 epochs
    random.shuffle(TRAIN_DATA)
    losses = {}
    
    with tqdm(total=len(TRAIN_DATA), desc=f"Epoch {itn + 1}") as progress_bar:
        for text, annotations in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], losses=losses)
            progress_bar.set_postfix(loss=losses['ner'], refresh=True)
            progress_bar.update(1)


# Save the model
nlp.to_disk('path_to_save_your_model')

Epoch 2: 100%|██████████| 7660/7660 [01:44<00:00, 72.97it/s, loss=3.69e-15]


In [4]:
text = "I would like a table at passims kitchen at 5 pm"

nlp = spacy.load('path_to_your_saved_model')
doc = nlp(text)
result = {ent.label_: ent.text for ent in doc.ents}
print(result)

OSError: [E050] Can't find model 'path_to_your_saved_model'. It doesn't seem to be a Python package or a valid path to a data directory.